In [1]:
# Import data from Excel sheet
import pandas as pd
df = pd.read_excel('/data_dzne_archiv2/Studien/Deep_Learning_Visualization/data/hippocampus_volume_relevance_ADNI2.xlsx', sheet_name='ADNI2_LRP_CMP')
#print(df)
sid = df['subject_ID']
grp = df['Group at scan date (1=CN, 2=EMCI, 3=LMCI, 4=AD, 5=SMC)']
age = df['Age at scan']
sex = df['Sex (1=female)']
tiv = df['TIV_CAT12']
field = df['MRI_Field_Strength']
amybin = df['Amy SUVR Pos']
grpbin = (grp > 1) # 1=CN, ...

In [2]:
# Scan for nifti file names
import glob
dataAD = sorted(glob.glob('/data_dzne_archiv2/Studien/Deep_Learning_Visualization/data/mwp1_MNI_ADNI/AD/*.nii.gz'))
dataLMCI = sorted(glob.glob('/data_dzne_archiv2/Studien/Deep_Learning_Visualization/data/mwp1_MNI_ADNI/LMCI/*.nii.gz'))
dataCN = sorted(glob.glob('/data_dzne_archiv2/Studien/Deep_Learning_Visualization/data/mwp1_MNI_ADNI/CN/*.nii.gz'))
dataFiles = dataAD + dataLMCI + dataCN
numfiles = len(dataFiles)
print('Found ', str(numfiles), ' nifti files')

Found  663  nifti files


In [3]:
import re
import numpy as np
from keras.utils import to_categorical
debug = False
cov_idx = [-1] * numfiles # list; array: np.full((numfiles, 1), -1, dtype=int)
print('Matching covariates for loaded files ...')
for i,id in enumerate(sid):
    p = [j for j,x in enumerate(dataFiles) if re.search('_%04d_' % id, x)] # translate ID numbers to four-digit numbers, get both index and filename
    if len(p)==0:
        if debug: print('Did not find %04d' % id) # did not find Excel sheet subject ID in loaded file selection
    else:
        if debug: print('Found %04d in %s: %s' % (id, p[0], dataFiles[p[0]]))
        cov_idx[p[0]] = i # store Excel index i for data file index p[0]
print('Checking for scans not found in Excel sheet: ', sum(x<0 for x in cov_idx))

labels = pd.DataFrame({'Group':grpbin}).iloc[cov_idx, :]
labels = to_categorical(np.asarray(labels)) # use grps to access original labels
grps = pd.DataFrame({'Group':grp, 'RID':sid}).iloc[cov_idx, :]

Using TensorFlow backend.


Matching covariates for loaded files ...
Checking for scans not found in Excel sheet:  0


In [4]:
#Load residualized data from disk
import h5py
hf = h5py.File('residuals_ADNI2_wb_mwp1_CAT12_MNI.hdf5', 'r')
hf.keys # read keys
images = np.array(hf.get('images'))
hf.close()

In [5]:
import tensorflow as tf
print(tf.__version__)

# disable tensorflow deprecation warnings
import logging
logging.getLogger('tensorflow').disabled=True

from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
    # device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
set_session(session)

1.15.5


2023-12-01 09:34:26.126259: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2023-12-01 09:34:26.137640: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3192500000 Hz
2023-12-01 09:34:26.138352: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xbe9bc80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-12-01 09:34:26.138376: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-12-01 09:34:26.141293: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-12-01 09:34:26.265739: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-01 09:34:26.26599

In [6]:
import nibabel as nib

# define FOV to reduce required memory size
x_range_from = 10; x_range_to = 110
y_range_from = 13; y_range_to = 133
z_range_from = 5; z_range_to = 105

data_overlay = sorted(glob.glob('Hippocampus_masks/aal_hippocampus.nii*'))

In [7]:
# Load CNN model from disk --> save predictions
from keras.models import load_model

batch_size = 20

for k in [0]: #range(20):
    mymodel = load_model('model_checkpoints/resmodel_wb_whole_ds.hdf5') #load_model('model_checkpoints/resmodel_wb_cv%d.best.hdf5' % (k+1))
    #mymodel.summary()
    pred = mymodel.predict(images, batch_size=batch_size)
    # only output prediction scores for AD output neuron
    results = pd.DataFrame({'SID':grps.RID, 'Group':grps.Group, 'pred':pred[:,1]})
    results.to_csv('results_wholeds_ADNI2_predictions.csv')

2023-12-01 09:34:28.796267: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2023-12-01 09:34:29.077898: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2023-12-01 09:34:30.109895: W tensorflow/stream_executor/cuda/redzone_allocator.cc:312] Not found: ./bin/ptxas not found
Relying on driver to perform ptx compilation. This message will be only logged once.


In [8]:
# Load CNN model from disk
from keras.models import load_model, Model
#!pip install innvestigate
import innvestigate
import innvestigate.utils as iutils
import numpy as np
from matplotlib import pyplot as plt
import scipy
import csv


# see https://github.com/albermax/innvestigate/blob/master/examples/notebooks/imagenet_compare_methods.ipynb for a list of alternative methods
methods = [ # tuple with method,     params,                  label
#            ("deconvnet",            {},                      "Deconvnet"),
#            ("guided_backprop",      {},                      "Guided Backprop"),
#            ("deep_taylor.bounded",  {"low": -1, "high": 1},  "DeepTaylor"),
#            ("input_t_gradient",     {},                      "Input * Gradient"),
#            ("lrp.z",                {},                      "LRP-Z"),
#            ("lrp.epsilon",          {"epsilon": 1},          "LRP-epsilon"),
#            ("lrp.alpha_1_beta_0",   {"neuron_selection_mode":"index"},     "LRP-alpha1beta0"),
        ("lrp.sequential_preset_a", {"neuron_selection_mode": "index", "epsilon": 1e-10}, "LRP-CMPalpha1beta0"), # LRP CMP rule taken from https://github.com/berleon/when-explanations-lie/blob/master/when_explanations_lie.py
]

hipp_nifti = nib.load(data_overlay[0])  # assume it is already 32bit float format
new_data = np.zeros((121, 145, 121), dtype=np.float32) 

for k in [0]: #range(20):
    mymodel = load_model('model_checkpoints/resmodel_wb_whole_ds.hdf5') #load_model('model_checkpoints/resmodel_wb_cv%d.best.hdf5' % (k+1))
    #mymodel.summary()
    #model_wo_softmax = iutils.keras.graph.model_wo_softmax(mymodel)  ## sometimes raises: ValueError: The name "dense_1" is used 2 times in the model. All layer names should be unique.
    #model_wo_softmax = Model(inputs=mymodel.inputs,
    #                          outputs=iutils.keras.graph.pre_softmax_tensors(mymodel.outputs),
    #                          name=(mymodel.name + '_wo_softmax')) 
    #model_wo_softmax.summary()
    mymodel.layers[-1].activation=tf.keras.activations.linear
    mymodel.save('tmp_wo_softmax.hdf5')
    model_wo_softmax = load_model('tmp_wo_softmax.hdf5')
    if (k==0):
        model_wo_softmax.summary()

    # create analyzer
    analyzers = []
    for method in methods:
        #analyzer = innvestigate.create_analyzer("deep_taylor.bounded", model_wo_softmax, **params )
        analyzer = innvestigate.create_analyzer(method[0], model_wo_softmax, **method[1])
        # Some analyzers require training.
        #   analyzer.fit(test_img, batch_size=30, verbose=1)
        #  analyzers.append(analyzer)
     
    for indx in range(len(grps)):
        test_img = images[indx]
        #test_orig = images_orig[indx]
        #print('test image for subject of binary group: %d' % test_Y[subj_idx, 1]) # first col will indicate CN, second col indicates MCI/AD
        #print('test image for subject of ADNI diagnosis: %d [1-CN, 3-LMCI, 4-AD]' % testgrps.Group.to_numpy(dtype=np.int)[subj_idx])

        ####print('test subject ID %s' % grps.RID.to_numpy(dtype=np.int)[indx])

        test_img = np.reshape(test_img, (1,)+ test_img.shape) # add first subj index again to mimic original array structure
        #test_orig = np.reshape(test_orig, (1,)+ test_orig.shape) # add first subj index again to mimic original array structure

        #for method,analyzer in zip(methods, analyzers):
        a = np.reshape(analyzer.analyze(test_img, neuron_selection=1), test_img.shape[1:4])
        #np.clip(a, a_min=0, a_max=None, out=a)
        #a = scipy.ndimage.filters.gaussian_filter(a, sigma=0.8) # smooth activity image
        #scale = np.quantile(a, 0.99) # no need for abs(a)
        #if scale!=0:  # ignore if relevance maps contains only zeros, output will be zero as well
        #    a = (a/scale)

        a = np.flip(a) # flip all positions
        a = np.transpose(a, (1, 2, 0)) # reorder dimensions from coronal view z*x*y back to x*y*z

        print('saving lrp_maps_ADNI2/lrp_map_grp%d_%04d_whole_ds_model.nii' % (grps.Group.iloc[indx], grps.RID.iloc[indx])) #'saving lrp_map_grp%d_%04d_cv%d.nii' % (grps.Group.iloc[indx], grps.RID.iloc[indx], (k+1)))
        new_data[x_range_from:x_range_to, y_range_from:y_range_to, z_range_from:z_range_to] = a
        nifti = nib.Nifti1Image(new_data, hipp_nifti.affine, hipp_nifti.header)
        nifti.to_filename('lrp_maps_ADNI2/lrp_map_grp%d_%04d_whole_ds_model.nii' % (grps.Group.iloc[indx], grps.RID.iloc[indx])) #'lrp_map_grp%d_%04d_cv%d.nii' % (grps.Group.iloc[indx], grps.RID.iloc[indx], (k+1)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 100, 100, 120, 5)  140       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 50, 50, 60, 5)     0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 50, 60, 5)     20        
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 50, 50, 60, 5)     680       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 25, 25, 30, 5)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 25, 25, 30, 5)     20        
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 25, 30, 5)     680       
__________

saving lrp_maps_ADNI2/lrp_map_grp4_0973_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_0994_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_1004_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_1255_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_6142_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4001_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4009_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4024_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4039_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4089_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4136_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4152_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4153_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4172_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4192_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4195_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp4_4201_whole_ds_model.n

saving lrp_maps_ADNI2/lrp_map_grp3_4311_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4630_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_0291_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_0296_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1226_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1106_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1118_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_5047_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4167_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1300_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_0709_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1187_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_0925_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1419_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4928_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_0200_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_1408_whole_ds_model.n

saving lrp_maps_ADNI2/lrp_map_grp3_4782_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4784_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4793_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4806_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4807_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4815_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4817_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4842_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4852_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4857_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4869_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4873_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4877_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4885_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4888_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4889_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp3_4892_whole_ds_model.n

saving lrp_maps_ADNI2/lrp_map_grp1_4104_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4119_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4120_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4121_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4125_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4139_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4148_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4150_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4151_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4155_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4158_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4164_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4173_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4174_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4176_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4177_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4179_whole_ds_model.n

saving lrp_maps_ADNI2/lrp_map_grp1_4900_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4921_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4951_whole_ds_model.nii
saving lrp_maps_ADNI2/lrp_map_grp1_4952_whole_ds_model.nii
